## Song recommendation with colaborative filtering for recommendation

Imporing all the necessary libraries and loading the data

In [2]:
import numpy as np

# Songs: Luis Fonsi - Despacito ft. Daddy Yankee; Gorillaz - Feel Good; U2 - One; Milky Chance - Stolen Dance;
# Major Lazer & DJ Snake - Lean On; Foo Fighters - Everlong; Coldplay - Viva la Vida .
P = [
    [0,  0, -1,  0, -1,  1,  1], # User 1
    [-1, 1,  1, -1,  0,  1,  1], # User 2
    [0,  1,  1,  0,  0, -1,  1], # User 3
    [-1, 1,  1,  0,  0,  1,  1], # User 4
    [0,  1,  1,  0,  0,  1,  1], # User 5
    [1, -1,  1,  1,  1, -1,  0], # User 6
    [-1, 1, -1,  0, -1,  0,  1], # User 7
    [0, -1,  0,  1,  1, -1, -1], # User 8
    [0,  0, -1,  1,  1,  0, -1]  # User 9
]

In [3]:
P = np.array(P)

# Parameters
reg = 0.1 # regularization parameter
f = 2 # number of factors
m, n = P.shape # where m = 9 and n = 7


# Random Initialization with random features
# X is (m x f) 9 x 2
# Y is (f x n) 2 x 7
X = 1 - 2 * np.random.rand(m, f)
Y = 1 - 2 * np.random.rand(f, n)
X *= 0.1
Y *= 0.1


# Alternating Ridge Regression
for _ in range(100):
    # Least-squares keeping Y fixed
    
    X = np.linalg.solve(
        np.dot(Y, Y.T) + reg * np.eye(f),
        np.dot(Y, P.T)
    ).T
    
    # Least-squares keeping X fixed
    Y = np.linalg.solve(
        np.dot(X.T, X) + reg * np.eye(f),
        np.dot(X.T, P)
    )
    
print("Alternating Ridge Regression:")
print(np.dot(X, Y))
print("\n")

Alternating Ridge Regression:
[[-0.47047911  0.42881358 -0.92322651 -0.40431668 -0.82818493  0.71377095
   0.35042332]
 [-0.57771727  1.11906181  0.94589704 -0.61386046 -0.41293492  0.72761603
   1.24992319]
 [-0.0832934   0.47515086  1.237768   -0.15067544  0.26037089  0.02607153
   0.61430677]
 [-0.47973313  0.97615801  0.95005937 -0.51903706 -0.29509201  0.59242729
   1.10280049]
 [-0.37594016  0.83720279  0.99806105 -0.42105292 -0.15760122  0.44610391
   0.96413724]
 [ 0.67254818 -0.68837737  1.05514838  0.59290532  1.10703237 -1.0013938
  -0.60521719]
 [-0.56690391  0.63195565 -0.70791877 -0.51001588 -0.88042517  0.83110411
   0.58167939]
 [ 0.65941678 -0.96038615  0.03268812  0.63788156  0.79442125 -0.91013218
  -0.98826789]
 [ 0.3261365  -0.67760331 -0.69495415  0.35562669  0.18635759 -0.39923655
  -0.76905842]]




In [7]:
# Re-initialize
X = 1 - 2 * np.random.rand(m, f)
Y = 1 - 2 * np.random.rand(f, n)
X *= 0.1
Y *= 0.1


# Alternating Weighted Ridge Regression
C = np.abs(P) # Will be 0 only when P[i,j] == 0.

for _ in range(100):
    
    # Each user u has a different set of weights Cu
    for u, Cu in enumerate(C):
        idCu = np.diag(Cu)
        
#         X[u] = np.dot(
#             np.linalg.inv(np.dot(np.dot(Y, idCu), Y.T) + np.eye(f) * reg),
#             np.dot(np.dot(Y, idCu), Cu)
#         )
        
        X[u] = np.linalg.solve(
            np.dot(np.dot(Y, idCu), Y.T) +  np.eye(f) * reg,
            np.dot(np.dot(Y, idCu), P[u])
        ).T
    
        
    for i, Ci in enumerate(C.T):
        idCi = np.diag(Ci)
        
#         Y[:, i] = np.dot(
#             np.linalg.inv(np.dot(np.dot(X.T, idCi), X) + np.eye(f) * reg),
#             np.dot(np.dot(X.T, idCi), Ci)
#         )

        Y[:, i] = np.linalg.solve(
            np.dot(np.dot(X.T, idCi), X) + np.eye(f) * reg,
            np.dot(np.dot(X.T, idCi), P[:, i])
        )
        
print("Alternating Weighted Ridge Regression:")
print(np.dot(X, Y))

Alternating Weighted Ridge Regression:
[[-0.87859229  0.75590936 -0.84723563 -0.90527377 -0.91301409  1.22456659
   0.77033933]
 [-0.96652919  1.12037758  0.83860292 -1.006266   -1.00817832  0.72382711
   1.10750305]
 [-0.32540386  0.55731589  1.38658534 -0.34525861 -0.34178468 -0.14502858
   0.53505233]
 [-0.96804632  1.12193896  0.83870996 -1.00783842 -1.00975825  0.72538898
   1.10906385]
 [-0.95828894  1.11192878  0.83821617 -0.99772664 -0.99959744  0.71527537
   1.0990542 ]
 [ 0.92243551 -0.80265171  0.83420692  0.95077283  0.95869316 -1.26620512
  -0.81690378]
 [-1.0270414   0.89416707 -0.92578394 -1.05860995 -1.06741719  1.40873074
   0.90998625]
 [ 0.95137069 -0.98077311 -0.07729113  0.98609632  0.99076854 -0.97584417
  -0.98024791]
 [ 0.89692883 -1.06910883 -0.95852321  0.93486169  0.93596393 -0.60823117
  -1.05423384]]
